# Customer Profitability
-----------------------------------------------------------------------------

## ! Important
For this code I divided the data into several csv file. You need to have ALL the csv files to run the code.

In [1]:
import pandas as pd ; import numpy as np ; import itertools ; import matplotlib.pyplot as plt
import string

In [30]:
products = list(string.ascii_uppercase)[0:12]
products.remove('J')
products

col = ['Running Number', 'Customer id']
col += [i for i in products]
col

['Running Number',
 'Customer id',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'K',
 'L']

### Step 1 : Data and cleaning (be ready it's very long)
- First we take care of the direct cost file
- Second Sale ledge
- Third Indirect cost

<u> File 1 : Direct cost </u>

In [31]:
direct_cost = pd.read_csv('Direct cost sheet.csv', delimiter=';', encoding= 'unicode_escape')
direct_cost.dropna(axis = 0, how = 'all', inplace = True)
direct_cost.dropna(axis = 1, how = 'all', inplace = True)
direct_cost.drop(direct_cost.head(5).index,inplace=True)

# Create a dataframe for the components used in direct cost
direct_driver_used = direct_cost.iloc[:,:len(products)+2].copy()
direct_driver_used.columns = col
direct_driver_used = direct_driver_used.apply(lambda x: x.astype(np.int))
direct_driver_used

,Running Number,Customer id,A,B,C,D,E,F,G,H,I,K,L
5,1,105,21,44,0,0,51,0,130,0,23,0,3
6,2,117,0,0,0,10,27,0,0,17,0,20,9
7,3,72,2,0,0,17,53,0,0,0,18,0,0
8,4,65,24,69,97,0,67,90,0,19,0,75,30
9,5,50,12,70,73,1,49,0,0,0,27,106,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3000,2996,52,5,0,0,20,0,33,15,0,0,44,0
3001,2997,121,21,5,0,12,38,0,14,0,0,71,0
3002,2998,138,6,0,26,6,0,135,38,0,0,0,42
3003,2999,149,17,0,45,0,24,71,0,28,10,0,18


In [32]:
#Dataframe for direct cost per component per order
direct_cost_order_v1 = direct_cost.iloc[:,len(products)+2:len(products)*2+2].copy()
info = pd.DataFrame(direct_driver_used[['Running Number', 'Customer id']])
direct_cost_order = info.join(direct_cost_order_v1)
direct_cost_order.columns = col

# Clean the data by removing the euro, comma and - signs
direct_cost_order= direct_cost_order.replace('\x80','',regex=True)
direct_cost_order= direct_cost_order.replace(',','.',regex=True)
direct_cost_order= direct_cost_order.replace('-',0,regex=True)

# Convert all product columns to floats
direct_cost_order[products] = direct_cost_order[products].apply(pd.to_numeric)
direct_cost_order[direct_cost_order.select_dtypes(include = object).columns] = direct_cost_order.select_dtypes(include = object).apply(lambda x: x.astype(np.int))
direct_cost_order

,Running Number,Customer id,A,B,C,D,E,F,G,H,I,K,L
5,1,105,7.58,4.46,0.00,0.00,9.54,0.00,6.97,0.00,8.13,0.00,7.23
6,2,117,0.00,0.00,0.00,3.80,12.73,0.00,0.00,10.85,0.00,7.52,19.31
7,3,72,7.10,0.00,0.00,10.11,7.17,0.00,0.00,0.00,7.02,0.00,0.00
8,4,65,8.96,2.86,12.49,0.00,6.75,53.71,0.00,8.61,0.00,11.52,16.26
9,5,50,5.36,5.02,18.67,4.34,12.44,0.00,0.00,0.00,11.07,6.54,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3000,2996,52,8.96,0.00,0.00,5.52,0.00,72.25,9.28,0.00,0.00,6.96,0.00
3001,2997,121,14.94,4.84,0.00,8.76,10.54,0.00,13.21,0.00,0.00,5.47,0.00
3002,2998,138,11.92,0.00,25.44,7.65,0.00,60.07,10.00,0.00,0.00,0.00,7.07
3003,2999,149,6.32,0.00,11.59,0.00,8.30,82.23,0.00,14.79,5.44,0.00,5.08


<u> File 2: Sales ledger </u>

In [33]:
sales_ledger = pd.read_csv('Sales ledger.csv', delimiter=';', encoding= 'unicode_escape')
sales_ledger.dropna(axis = 0, how = 'all', inplace = True)
sales_ledger.dropna(axis = 1, how = 'all', inplace = True)
sales_ledger.drop(sales_ledger.tail(1).index,inplace=True)
sales_ledger.drop(sales_ledger.head(2).index,inplace=True)
sales_ledger

,Unnamed: 0,Unnamed: 1,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,...,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28
5,1,105,17,0,63,0,28,0,95,0,..., -," 49,00", -," 16,00", -," 6,00", -, -," 17,00"," 18,00"
6,2,117,16,0,0,17,0,89,0,11,..., -, -," 5,00", -," 103,00", -," 18,00"," 15,00", -, -
7,3,72,30,31,0,0,25,63,74,55,...," 9,00", -, -," 15,00"," 167,00"," 14,00"," 15,00"," 16,00", -," 20,00"
8,4,65,4,0,24,0,44,92,0,58,..., -," 49,00", -," 19,00"," 101,00", -," 17,00", -, -, -
9,5,50,18,0,0,2,0,0,43,0,..., -, -," 17,00", -, -," 30,00", -," 19,00"," 16,00", -
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3000,2996,52,4,9,0,0,0,43,0,0,...," 7,00", -, -, -," 139,00", -, -," 20,00"," 16,00", -
3001,2997,121,20,28,72,0,0,0,28,0,...," 8,00"," 20,00", -, -, -," 36,00", -, -," 18,00", -
3002,2998,138,0,41,0,0,10,0,82,0,...," 10,00", -, -," 15,00", -," 22,00", -, -," 19,00", -
3003,2999,149,11,30,38,0,0,0,11,21,...," 6,00"," 33,00", -, -, -," 20,00"," 16,00"," 15,00", -," 18,00"


In [35]:
# Create a dataframe for the units sold
sale_product = sales_ledger.iloc[:,:len(products)+2].copy()
sale_product.columns = col
sale_product = sale_product.apply(lambda x: x.astype(np.int))
sale_product

,Running Number,Customer id,A,B,C,D,E,F,G,H,I,K,L
5,1,105,17,0,63,0,28,0,95,0,0,90,34
6,2,117,16,0,0,17,0,89,0,11,3,0,0
7,3,72,30,31,0,0,25,63,74,55,16,0,35
8,4,65,4,0,24,0,44,92,0,58,0,0,0
9,5,50,18,0,0,2,0,0,43,0,17,66,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3000,2996,52,4,9,0,0,0,43,0,0,13,101,0
3001,2997,121,20,28,72,0,0,0,28,0,0,36,0
3002,2998,138,0,41,0,0,10,0,82,0,0,13,0
3003,2999,149,11,30,38,0,0,0,11,21,8,0,30


In [38]:
#Dataframe for direct cost per component per order
sale_price_unit_v1 = sales_ledger.iloc[:,len(products)+2:len(products)*2+2].copy()
info = pd.DataFrame(sale_product[['Running Number', 'Customer id']])
sale_price_unit = info.join(sale_price_unit_v1)
sale_price_unit.columns = col

# Clean the data by removing the euro, comma and - signs
sale_price_unit= sale_price_unit.replace('\x80','',regex=True)
sale_price_unit= sale_price_unit.replace(',','.',regex=True)
sale_price_unit= sale_price_unit.replace('-',0,regex=True)

# Convert all product columns to floats
sale_price_unit[products] = sale_price_unit[products].apply(pd.to_numeric)
sale_price_unit[sale_price_unit.select_dtypes(include = object).columns] = sale_price_unit.select_dtypes(include = object).apply(lambda x: x.astype(np.int))
sale_price_unit


,Running Number,Customer id,A,B,C,D,E,F,G,H,I,K,L
5,1,105,327.0,16.0,0.0,49.0,0.0,16.0,0.0,6.0,0.0,0.0,17.0
6,2,117,136.0,19.0,0.0,0.0,5.0,0.0,103.0,0.0,18.0,15.0,0.0
7,3,72,329.0,18.0,9.0,0.0,0.0,15.0,167.0,14.0,15.0,16.0,0.0
8,4,65,222.0,18.0,0.0,49.0,0.0,19.0,101.0,0.0,17.0,0.0,0.0
9,5,50,146.0,19.0,0.0,0.0,17.0,0.0,0.0,30.0,0.0,19.0,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3000,2996,52,170.0,18.0,7.0,0.0,0.0,0.0,139.0,0.0,0.0,20.0,16.0
3001,2997,121,184.0,15.0,8.0,20.0,0.0,0.0,0.0,36.0,0.0,0.0,18.0
3002,2998,138,146.0,0.0,10.0,0.0,0.0,15.0,0.0,22.0,0.0,0.0,19.0
3003,2999,149,149.0,18.0,6.0,33.0,0.0,0.0,0.0,20.0,16.0,15.0,0.0
